In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
zarif98sjs_gait_in_parkinsons_disease_path = kagglehub.dataset_download('zarif98sjs/gait-in-parkinsons-disease')

print('Data source import complete.')


In [ ]:
##pip install numpy==1.22.0

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
import random

import os


In [ ]:
input_dir = '/kaggle/input/gait-in-parkinsons-disease'
len(os.listdir(input_dir))

In [ ]:
features = ['Time', 'L1' , 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Total_Force_Left', 'Total_Force_Right']

In [ ]:
!mkdir CSV

In [ ]:
i = 0
for file in os.listdir(input_dir):
    i = i+1
    ##print(i)
    if file[0:2]=='Ga' or file[0:2]=='Ju' or file[0:2] == 'Si':
        pass
        ##print(file)
        df = pd.read_csv(input_dir + '/' + file, header = None, sep = '\t', on_bad_lines = 'skip')

        df.columns = features

        name = 'CSV/' + file.split('.')[0] + '.csv'
        df.to_csv(name, index = None)

print(i)

In [ ]:
demographics = pd.read_csv(input_dir + '/demographics.txt', delim_whitespace = True)
sub_id = demographics.ID.to_list()
sub_names = []
for name in os.listdir('CSV'):
    sub_name = name.split('_')[0]
    sub_names.append(sub_name)

print("Subjects Count = ", len(sub_id))
print("Files Count = ", len(sub_names))

In [ ]:
!mkdir train
!mkdir validate
!mkdir test

In [ ]:
count = len(os.listdir('CSV'))
train_count = int(70/100*count)
valid_count = int(15/100*count)
test_count = count - train_count - valid_count

print("Training count = ", train_count)
print("Validate count = ", valid_count)
print("Test count = ", test_count)

In [ ]:
def load():
    count = len(os.listdir('CSV'))
    train_count = int(70/100*count)
    valid_count = int(15/100*count)
    test_count = count - train_count - valid_count

    train_X = list()
    train_Y = list()

    valid_X = list()
    valid_Y = list()

    test_X = list()
    test_Y = list()

    i = 0

    for name in os.listdir('CSV'):
        ##if(i>10):
        ##    break
        ##print(name)
        sub_id = name.split('_')[0]
        sub_class = demographics[demographics['ID']==sub_id]['Group']
        sub_class -= 1
        sub_class.to_string(index=False).strip()
        ##(sub_class)

        dataframe = pd.read_csv('CSV/' + name)
        full_size = 400
        skip = 50

        X = list()
        ##Y = list()
        ##Y.append(sub_class)

        max_j = 150
        for j in range(0, dataframe.shape[0], skip):
            if dataframe.shape[0]>=full_size+j:
                temp = list()
                for feature in features:
                    temp.append(dataframe.iloc[j:j+full_size, ][feature].to_numpy())
                ##X.append(temp)

            if(i<train_count):
                train_X.append(temp)
                train_Y.append(sub_class)
                        ##print(i, name, "train")
            elif(train_count<=i<valid_count+train_count):
                valid_X.append(temp)
                valid_Y.append(sub_class)
                        ##print(i, name, "valid")
            else:
                test_X.append(temp)
                test_Y.append(sub_class)
                    ##print(i, name, "test")
        print(i, name)
        i+=1

    train_X = np.array(train_X)
    train_Y = tf.keras.utils.to_categorical(train_Y)

    valid_X = np.array(valid_X)
    valid_Y = tf.keras.utils.to_categorical(valid_Y)

    test_X = np.array(test_X)
    test_Y = tf.keras.utils.to_categorical(test_Y)

    """valid_X = np.destack(valid_X)
    valid_Y = np.destack(valid_Y)

    test_X = np.destack(test_X)
    test_Y = np.destack(test_Y)"""

    return train_X, train_Y, valid_X, valid_Y, test_X, test_Y

In [ ]:
train_X, train_Y, valid_X, valid_Y, test_X, test_Y = load()
print("Training Data = ", train_X.shape)
print("Training Class = ", train_Y.shape)
print("Valid Data = ", valid_X.shape)
print("Valid Class = ", valid_Y.shape)
print("Test Data = ", test_X.shape)
print("Test Class = ", test_Y.shape)

In [ ]:
train_X.reshape(train_X.shape[0], 19, train_X.shape[2], 1)
valid_X.reshape(valid_X.shape[0], 19, valid_X.shape[2], 1)
test_X.reshape(test_X.shape[0], 19, test_X.shape[2], 1)

In [ ]:
##pip install keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape = (19, 400, 1)))
model.add(Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))

#------------------------------------
# Conv Block 3: 64 Filters, MaxPool.
#------------------------------------
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#------------------------------------
# Flatten the convolutional features.
#------------------------------------
model.add(Flatten())
model.add(Dense(128, activation='elu'))
model.add(Dense(2, activation='softmax'))


In [ ]:
model.summary()

In [ ]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate = 0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy',
                         tf.keras.metrics.Precision(name = 'precision'),
                         tf.keras.metrics.TruePositives(name ='tp'),
                         tf.keras.metrics.FalseNegatives(name = 'fn'),
                         tf.keras.metrics.FalsePositives(name = 'fp'),
                         tf.keras.metrics.TrueNegatives(name = 'tn'),
                         tf.keras.metrics.Recall(name='recall'),
                         tf.keras.metrics.AUC(name='auc')])
model.summary()

In [ ]:
from keras import callbacks
early_stop = callbacks.EarlyStopping(monitor = "accuracy", mode = "auto", restore_best_weights = True)

In [ ]:
history = model.fit(train_X,
                   train_Y,
                   batch_size = 32,
                   epochs = 3,
                   verbose = 1,
                   validation_data = (valid_X, valid_Y),
                   callbacks = [early_stop])

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

val_predictions = model.predict(valid_X)
val_predictions_classes = np.argmax(val_predictions, axis=1)
val_true_classes = np.argmax(valid_Y, axis=1)

precision = precision_score(val_true_classes, val_predictions_classes, average='weighted')
recall = recall_score(val_true_classes, val_predictions_classes, average='weighted')
f1 = f1_score(val_true_classes, val_predictions_classes, average='weighted')

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')